## Import required modules

In [2]:
# Bash: pip install --upgrade python-docx

In [83]:
# Create 'docs' directory if it doesn't exist
import os 

os.makedirs('docs', exist_ok=True)

print("Now load all your files to the docs folder...")

Now load all your files to the docs folder...


In [95]:
import os 
import pandas as pd
from PyPDF2 import PdfReader
import openpyxl
import re
from docx import Document


def extract_sentences(file_path):
    """Extract sentences with proper resource cleanup"""
    content_blocks = []
    
    try:
        if file_path.lower().endswith('.xlsx'):
            # Use pandas ExcelFile for better resource management
            with pd.ExcelFile(file_path, engine='openpyxl') as xls:
                for sheet_name in xls.sheet_names:
                    df = xls.parse(sheet_name)
                    for row_idx, row in df.iterrows():
                        for col_idx, value in enumerate(row):
                            cell_text = str(value)
                            sentences = re.split(r'(?<=[.!?])\s+', cell_text)
                            for sentence in sentences:
                                if sentence := sentence.strip():
                                    content_blocks.append({
                                        'source_type': 'worksheet',
                                        'source_name': sheet_name,
                                        'content': sentence,
                                        'location': f"Row {row_idx+1}, Col {df.columns[col_idx]}"
                                    })
        elif file_path.lower().endswith('.pdf'):
            # PDF handling with guaranteed closure
            with open(file_path, 'rb') as f:
                pdf = PdfReader(f)
                for page_num, page in enumerate(pdf.pages, 1):
                    page_text = page.extract_text()
                    if page_text:
                        sentences = re.split(r'(?<=[.!?])\s+', page_text)
                        for sentence in sentences:
                            if sentence := sentence.strip():
                                content_blocks.append({
                                    'source_type': 'page',
                                    'source_name': f"Page {page_num}",
                                    'content': sentence,
                                    'location': None
                                })
        elif file_path.lower().endswith('.docx'):
            # Word document handling
            with open(file_path, 'rb') as f:
                doc = Document(f)
                for para_num, paragraph in enumerate(doc.paragraphs, 1):
                    paragraph_text = paragraph.text
                    if paragraph_text:
                        sentences = re.split(r'(?<=[.!?])\s+', paragraph_text)
                        for sentence in sentences:
                            if sentence := sentence.strip():
                                content_blocks.append({
                                    'source_type': 'paragraph',
                                    'source_name': f"Paragraph {para_num}",
                                    'content': sentence,
                                    'location': None
                                })
    
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
    
    return content_blocks

In [96]:
def find_keyword_matches(content_blocks, keywords):
    """Find exact keyword-containing sentences"""
    matches = []
    
    for block in content_blocks:
        lower_content = block['content'].lower()
        found_keywords = [kw for kw in keywords if kw.lower() in lower_content]
        
        if found_keywords:
            for keyword in found_keywords:
                matches.append({
                    'File Path': block.get('file_path', ''),
                    'Source Type': block['source_type'],
                    'Source Name': block['source_name'],
                    'Location': block['location'],
                    'Keyword': keyword,
                    'Exact Sentence': block['content']
                })
    
    return matches

In [114]:
def generate_report(folder_path, keywords):
    """Generate report with guaranteed file closure"""
    report = []
    
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(('.xlsx', '.pdf', '.docx')):
                file_path = os.path.join(root, file)
                content_blocks = extract_sentences(file_path)
                for block in content_blocks:
                    block['file_path'] = file_path
                matches = find_keyword_matches(content_blocks, keywords)
                report.extend(matches)
    
    return pd.DataFrame(report)

In [123]:
# Configuration
folder_path = 'docs' # Update this path
keywords = ['gender', 'transgender', 'transmen', 'transwomen', 'lgbtq', 'lgbt', ' dei ', 'diversity', 'equity', 'inclusion', 
            ' gbv ', 'trans-gender', 'trans-women', 'trans-men', 'disparity', 'pregnant people', 'identity', 'inclusivity', 
            'binary', 'non-binary', 'prejudice', 'pronouns', ' race ', 'stereotype', ' tgw ', ' tg ', 'transgender women', 
            ' trans ', 'protecting women', 'key pops', 'key populations', ' mat ', 'hormone', ' dreams ', 'abortion', 'per diem',
            ' fsw ', 'female sex worker', 'food', 'lunch', 'refreshment', 'allowance', ' meal', ' tea ', 'snack', 'gratuity', 'stipend', 'honorarium', 'incentive', 'advocacy', 'underrepresented', 'underserved']  # Add more keywords as needed
# Generate and save report
df = generate_report(folder_path, keywords)
df

c:\Users\uat6\Desktop\eo-compliance\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
c:\Users\uat6\Desktop\eo-compliance\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
c:\Users\uat6\Desktop\eo-compliance\.venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\uat6\Desktop\eo-compliance\.venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,File Path,Source Type,Source Name,Location,Keyword,Exact Sentence
0,docs\GGH002543_Kisumu County\Budget Narrative-...,paragraph,Paragraph 45,None,gratuity,This constitutes gratuity at 31% of the basic ...
1,docs\GGH002543_Kisumu County\Budget Narrative-...,paragraph,Paragraph 45,None,gratuity,The 31% gratuity is as guided by the directora...
2,docs\GGH002543_Kisumu County\Budget Narrative-...,paragraph,Paragraph 45,None,gratuity,11 (315) on payment of service gratuity to emp...
3,docs\GGH002543_Kisumu County\Budget Narrative-...,paragraph,Paragraph 65,None,stipend,Stipend for lay workers: -
4,docs\GGH002543_Kisumu County\Clean_Kisumu DHS_...,paragraph,Paragraph 41,None,inclusion,The program endeavored to develop a reasonable...
...,...,...,...,...,...,...
2409,docs\GH002458_Muranga County\Muranga County_ T...,worksheet,Tujitegemee Detailed Budget,"Row 71, Col Unnamed: 4",stipend,Stipend Amount
2410,docs\GH002458_Muranga County\Muranga County_ T...,worksheet,Tujitegemee Detailed Budget,"Row 71, Col Unnamed: 5",allowance,Allowances
2411,docs\GH002458_Muranga County\Muranga County_ T...,worksheet,Tujitegemee Detailed Budget,"Row 74, Col Murang'a County Department Of Heal...",stipend,Sub Total- Stipends Anciliary Workers
2412,docs\GH002458_Muranga County\Muranga County_ T...,worksheet,Tujitegemee Detailed Budget,"Row 75, Col Murang'a County Department Of Heal...",stipend,Total- Salaries & Wages/Stipends


In [124]:
df.to_csv(f"{folder_path}_keyword_summary.csv", index=False)

In [125]:
print(df.columns)

Index(['File Path', 'Source Type', 'Source Name', 'Location', 'Keyword',
       'Exact Sentence'],
      dtype='object')


In [126]:
df.rename(columns={'File Path': 'Path', 
                   'Exact Sentence': 'Content'}, inplace=True
          )
df.head()

,Path,Source Type,Source Name,Location,Keyword,Content
0,docs\GGH002543_Kisumu County\Budget Narrative-...,paragraph,Paragraph 45,None,gratuity,This constitutes gratuity at 31% of the basic ...
1,docs\GGH002543_Kisumu County\Budget Narrative-...,paragraph,Paragraph 45,None,gratuity,The 31% gratuity is as guided by the directora...
2,docs\GGH002543_Kisumu County\Budget Narrative-...,paragraph,Paragraph 45,None,gratuity,11 (315) on payment of service gratuity to emp...
3,docs\GGH002543_Kisumu County\Budget Narrative-...,paragraph,Paragraph 65,None,stipend,Stipend for lay workers: -
4,docs\GGH002543_Kisumu County\Clean_Kisumu DHS_...,paragraph,Paragraph 41,None,inclusion,The program endeavored to develop a reasonable...


In [127]:
def extract_partner_filename(df):
    """
    Extract Partner and File Name from Path column
    
    Args:
        df: DataFrame with 'Path' column
    
    Returns:
        DataFrame with added 'Partner' and 'File Name' columns
    """
    
    def parse_path(path):
        if pd.isna(path):
            return None, None
        
        # Split by backslash to get path components
        parts = path.split('\\')
        
        if len(parts) < 2:
            return None, None
        
        # Extract partner from the folder structure (e.g., "GH002250_LVCT_STARS")
        partner_folder = parts[1] if len(parts) > 1 else None
        
        # Extract partner name (everything after the underscore)
        partner = None
        if partner_folder and '_' in partner_folder:
            partner = partner_folder.split('_', 1)[1]  # Get everything after first underscore
        
        # Extract filename (last part of path before any additional info)
        filename = None
        if len(parts) > 2:
            filename = parts[2]
        
        return partner, filename
    
    # Apply the extraction function
    df[['Partner', 'File Name']] = df['Path'].apply(
        lambda x: pd.Series(parse_path(x))
    )
    
    return df

In [128]:
df_copy = extract_partner_filename(df)[['Partner', 'File Name', 'Source Type', 'Source Name', 'Location', 'Keyword', 'Content']]
df_copy.head()

,Partner,File Name,Source Type,Source Name,Location,Keyword,Content
0,Kisumu County,Budget Narrative- KISUMU CHMT 20250629 20.36pm...,paragraph,Paragraph 45,None,gratuity,This constitutes gratuity at 31% of the basic ...
1,Kisumu County,Budget Narrative- KISUMU CHMT 20250629 20.36pm...,paragraph,Paragraph 45,None,gratuity,The 31% gratuity is as guided by the directora...
2,Kisumu County,Budget Narrative- KISUMU CHMT 20250629 20.36pm...,paragraph,Paragraph 45,None,gratuity,11 (315) on payment of service gratuity to emp...
3,Kisumu County,Budget Narrative- KISUMU CHMT 20250629 20.36pm...,paragraph,Paragraph 65,None,stipend,Stipend for lay workers: -
4,Kisumu County,Clean_Kisumu DHS_Project Narrative_ 13July2025...,paragraph,Paragraph 41,None,inclusion,The program endeavored to develop a reasonable...


In [129]:
# Validation Tests (Win + .)
assert df_copy['Partner'].isnull().sum() == 0, "❌ Missing values found in 'Partner' column"
assert all(df_copy['Partner'].str.contains(r'[^\\/]+', regex=True)), "🤣 'Partner' column contains invalid values"

print("✅ Validation passed: All extracted filenames are correct!")

✅ Validation passed: All extracted filenames are correct!


In [130]:
# Save to excel
df_copy.to_excel(f"{folder_path} Keywords Check.xlsx", 
                 index=True,
                 index_label='No.')